## Part 04 머신러닝 - 1장 지도 학습 모형

### 9절: 앙상블
- 사이킷런에서 지원하는 앙상블 모형은 배깅(Bagging), 랜덤포레스트(Random Forest), 에이다부스트(AdaBoost), 그래디언트부스팅(Gradient Boosting) 등이 있음
- +) 사이킷런에서는 지원하지 않지만 별도의 패키지에서 사용 가능한 앙상블 모형인 XGBoost와 LightGBM의 모형객체 생성하는 방법

<br>

#### **1) 배깅 (Bagging)**

- 배깅은 사이킷런 서브패키지 ```ensemble``` 내 클래스 ```BaggingClassifer()```, ```BaggingRegressor()``` 등을 통해 모형객체를 생성할 수 있음
- BaggingClassifier는 이진분류와 다지분류, BaggingRegressor는 연속형 타겟변수에 사용됨  

#### **1-1) BaggingClassifier**

- 이진분류와 다지분류
- ```sklearn.ensemble.BaggingClassifer(base_estimator = None, n_estimators = 10, max_samples = 1.0, max_features = 1.0, ...)```  
   - **base_estimator**: 데이터의 랜덤 부분집합에 적합할 분류기. None(default)면 DecisionTressClassifer()
   - **n_estimators**: 앙상블할 분류기의 수 (default = 10)
   - **max_samples**: 각 분류기를 학습하기 위해 X에서 추출할 샘플의 수 (default = 1.0)
   - **max_features**: 각 분류기를 학습하기 위해 X에서 사용한 컬럼의 수

#### Q.
사이킷런 패키지 내 breast_cancer 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 BaggingClassifier()를 통해 이진분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 AUC 사용하기)

In [1]:
# 클래스와 함수 불러오기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import train_test_split

In [2]:
# breast_cancer 데이터 가져오기
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [3]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2205)

In [4]:
# 모형객체 생성
dtr = DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 10)
bag_bin = BaggingClassifier(base_estimator = dtr,
                            n_estimators = 500,
                            random_state = 2022)

In [5]:
# 모델학습
model_bag_bin = bag_bin.fit(X_train, y_train)
model_bag_bin

C:\Users\kbr_u\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


BaggingClassifier(base_estimator=DecisionTreeClassifier(max_depth=3,
                                                        min_samples_leaf=10),
                  n_estimators=500, random_state=2022)

In [6]:
# ROC
from sklearn.metrics import roc_curve, auc
y_score = model_bag_bin.predict_proba(X_test)[:, 1]

In [7]:
fpr, tpr, thresholds = roc_curve(y_test, y_score)
print(fpr)
print(tpr)
print(thresholds)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.04761905 0.04761905 0.14285714 0.14285714
 0.95238095 1.        ]
[0.         0.01388889 0.05555556 0.08333333 0.125      0.34722222
 0.38888889 0.97222222 0.97222222 0.98611111 0.98611111 1.
 1.         1.        ]
[           inf 9.92764022e-01 9.92689948e-01 9.92597355e-01
 9.92585450e-01 9.90710704e-01 9.90015109e-01 3.55463969e-01
 2.61973429e-01 2.59376355e-01 1.83090882e-01 1.59505066e-01
 1.94396858e-03 8.48397484e-04]


In [8]:
# AUC
AUC = auc(fpr, tpr)
AUC

0.9973544973544974

<br>

#### Q.
사이킷런 패키지 내 iris 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 BaggingClassifier()를 통해 다지분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 macro f1-score 사용하기)

In [9]:
# 클래스 함수 불러오기
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

In [10]:
# iris 데이터 가져오기
from sklearn.datasets import load_iris
iris = load_iris()
data = iris.data
target = iris.target

In [11]:
# 데이터 분할하기
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2, 
                                                    stratify = target,
                                                    random_state = 2205)

In [12]:
# 모형객체 생성
dtr = DecisionTreeClassifier(max_depth = 3, min_samples_leaf = 10)
bag_multi = BaggingClassifier(base_estimator = dtr,
                              n_estimators = 500,
                              random_state = 2022)

In [13]:
# 모엘학습
model_bag_multi = bag_multi.fit(X_train, y_train)

C:\Users\kbr_u\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [14]:
# macro f1-score
from sklearn.metrics import f1_score
y_pred = model_bag_multi.predict(X_test)

In [15]:
macro_f1 = f1_score(y_test, y_pred, average='macro')
macro_f1

0.9665831244778613

<br><br>

#### **1-2) BaggingRegressor**

- 연속형 타겟변수
- ```sklearn.ensemble.BaggingRegressor(base_estimator = None, n_estimators = 10, max_samples = 1.0, max_features = 1.0, ...)```  
   - **base_estimator**: 데이터의 랜덤 부분집합에 적합할 분류기. None(default)면 DecisionTressClassifer()
   - **n_estimators**: 앙상블할 분류기의 수 (default = 10)
   - **max_samples**: 각 분류기를 학습하기 위해 X에서 추출할 샘플의 수 (default = 1.0)
   - **max_features**: 각 분류기를 학습하기 위해 X에서 사용한 컬럼의 수

#### Q.
사이킷런 패키지 내 diabetes 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 BaggingRegressor()를 통해 연속형 예측 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, 평가지표는 RMSE 사용하기)

In [16]:
# 클래스 함수 불러오기
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

In [17]:
# diabetes 데이터 가져오기
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = diabetes.data
target = diabetes.target

In [18]:
# 데이터 분할하기
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2, 
                                                    random_state = 2205)

In [19]:
# 모델객체 생성
dtr = DecisionTreeRegressor(max_depth = 3, min_samples_leaf = 10)
bag_conti = BaggingRegressor(base_estimator = dtr,
                             n_estimators = 500,
                             random_state = 2022)

In [20]:
# 모델학습
model_bag_conti = bag_conti.fit(X_train, y_train)

C:\Users\kbr_u\anaconda3\Lib\site-packages\sklearn\ensemble\_base.py:156: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


In [21]:
# RMSE
from sklearn.metrics import mean_squared_error
y_pred = model_bag_conti.predict(X_test)

In [22]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
rmse

62.45208491746776

<br><br><br>
<hr>

#### **2) 랜덤포레스트 (RandomForest)**

- 랜덤포레스트는 사이킷런 서브패키지 ```ensemble``` 내 클래스 ```RandomForestClassifier()```, ```RandomForestRegressor()``` 등을 통해 모형객체를 생성할 수 있음
- RandomForestClassifier는 이진분류와 다지분류, RandomForestRegressor는 연속형 타겟변수에 사용됨  

#### **2-1) RandomForestClassifier**

- 이진분류와 다지분류
- ```sklearn.ensemble.RandomForestClassifier(n_estimators = 100, criterion = 'gini', max_depth = None, min_samples_leaf = 1, max_features = 'sqrt', ...)```  
   - **n_estimators**: 트리의 수 (default = 100)
   - **criterion**: 분할에 기준이 되는 지표
      - 'gini' (default)
      - 'entropy'
      - 'log_loss'
   - **max_depth**: 트리의 최대 깊이 (default = None)
   - **min_samples_leaf**: 리프 노드에 있어야 하는 최소 샘플 수 (default = 1)
   - **max_features**: 최적의 분할을 찾을 때 고려할 컬럼 수 (default= 'sqrt')
      - 'auto'
      - 'sqrt': sqrt(컬럼수)
      - 'log2': log2(컬럼수),
      - 'None': 컬럼수 또는 원하는 컬럼 수를 정수로 직접 입력 가능

<br>

#### Q.
사이킷런 패키지 내 breast_cancer 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 RandomForestClassifier()를 통해 이진분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 AUC 사용하기)

In [23]:
# 클래스 함수 불러오기
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [25]:
# breast_cancer 데이터 가져오기
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [26]:
# 데이터 분할하기
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2205)

In [27]:
# 모형객체 생성
rf_bin = RandomForestClassifier(n_estimators = 500,
                                max_depth = 3,
                                min_samples_leaf = 10,
                                max_features = 'sqrt',
                                random_state = 2022)

In [28]:
# 모델학습
model_rf_bin = rf_bin.fit(X_train, y_train)

In [29]:
# ROC
from sklearn.metrics import roc_curve, auc
y_score = model_rf_bin.predict_proba(X_test)[:, 1]

In [30]:
fpr, tpr, thresholds = roc_curve(y_test, y_score)
print(fpr)
print(tpr)
print(thresholds)

[0.         0.         0.         0.02380952 0.02380952 1.        ]
[0.         0.01388889 0.98611111 0.98611111 1.         1.        ]
[       inf 0.98976089 0.39041401 0.37395488 0.36962848 0.00407486]


In [31]:
# AUC
AUC = auc(fpr, tpr)
AUC

0.9996693121693121

<br>

#### Q.
사이킷런 패키지 내 iris 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 RandomForestClassifer()를 통해 다지분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 macro f1-score 사용하기)

In [32]:
# 클래스 함수 불러오기
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [33]:
# iris 데이터 가져오기
from sklearn.datasets import load_iris
iris = load_iris()
data = iris.data
target = iris.target

In [35]:
# 데이터 분할하기
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2, 
                                                    stratify = target,
                                                    random_state = 2205)

In [36]:
# 모델객체 생성
rf_multi = RandomForestClassifier(n_estimators = 500,
                                  max_depth = 3,
                                  min_samples_leaf = 15,
                                  max_features = 'sqrt',
                                  random_state = 2022)

In [37]:
# 모델학습
model_rf_multi = rf_multi.fit(X_train, y_train)

In [39]:
# macro f1-score
from sklearn.metrics import f1_score
y_pred = model_rf_multi.predict(X_test)

In [40]:
macro_f1 = f1_score(y_test, y_pred, average = 'macro')
macro_f1

0.9665831244778613

<br><br>

#### **2-2) RandomForestRegressor**

- 연속형 타겟변수
- ```sklearn.ensemble.RandomForestRegressor(n_estimators = 100, criterion = 'squared_error', max_depth = None, min_samples_leaf = 1, max_features = 1.0, ...)```  
   - **n_estimators**: 트리의 수 (default = 100)
   - **criterion**: 분할에 기준이 되는 지표
      - 'squared_error' (default)
      - 'abolute_error'
   - **max_depth**: 트리의 최대 깊이 (default = None)
   - **min_samples_leaf**: 리프 노드에 있어야 하는 최소 샘플 수 (default = 1)
   - **max_features**: 최적의 분할을 찾을 때 고려할 컬럼 수 (default= 1.0)
      - 'auto'
      - 'sqrt': sqrt(컬럼수)
      - 'log2': log2(컬럼수),
      - 'None': 컬럼수 또는 원하는 컬럼 수를 정수로 직접 입력 가능

#### Q.
사이킷런 패키지 내 diabetes 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 RandomForestRegressor()를 통해 연속형 예측 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고 평가지표는 RMSE 사용하기)

In [42]:
# 클래스 함수 불러오기
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [43]:
# diabetes 데이터 가져오기
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = diabetes.data
target = diabetes.target

In [44]:
# 데이터 분할하기
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    random_state = 2205)

In [45]:
# 모델객체 생성
rf_conti = RandomForestRegressor(n_estimators = 500,
                                 max_depth = 3,
                                 min_samples_leaf = 10,
                                 max_features = 3,
                                 random_state = 2022)

In [46]:
# 모델학습
model_rf_conti = rf_conti.fit(X_train, y_train)

In [47]:
# RMSE
from sklearn.metrics import mean_squared_error
y_pred = model_rf_conti.predict(X_test)

In [49]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
rmse

63.50376483948605

<br><br><br>
<hr>

#### **3) AdaBoost**

- AdaBoost는 사이킷런 서브패키지 ```ensemble``` 내 클래스 ```AdaBoostClassifier()```, ```AdaBoostRegressor()``` 등을 통해 모형객체를 생성할 수 있음
- AdaBoostClassifier는 이진분류와 다지분류, AdaBoostRegressor는 연속형 타겟변수에 사용됨  

#### **3-1) AdaBoostClassifier**

- 이진분류와 다지분류
- ```sklearn.ensemble.AdaBoostClassifier(base_estimator = None, n_estimators = 50, learning_rate = 1.0, ...)```  
   - **base_estimator**: 부스팅에 사용될 기본 분류기. None(default)면 DecisionTreeClassifier(max_depth = 1)
   - **n_estimators**: 부스팅이 종료되는 최대 분류기 수 (default = 50)
   - **learning_rate**: 부스팅 반복시 각 분류기에 적용되는 가중치 (default = 1.0)

<br>

#### Q.
사이킷런 패키지 내 breast_cancer 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 AdaBoostClassifier()를 통해 이진분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 AUC 사용하기)

In [50]:
# 클래스 함수 불러오기
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

In [51]:
# breast_cancer 데이터 가져오기
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [52]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2, 
                                                    stratify = target,
                                                    random_state = 2205)

In [53]:
# 모델객체 생성
ada_bin = AdaBoostClassifier(n_estimators = 100,
                             learning_rate = 0.5,
                             random_state = 2022)

In [54]:
# 모델학습
model_ada_bin = ada_bin.fit(X_train, y_train)

In [55]:
# ROC
from sklearn.metrics import roc_curve, auc
y_score = model_ada_bin.predict_proba(X_test)[:, 1]

In [56]:
fpr, tpr, thresholds = roc_curve(y_test, y_score)
print(fpr)
print(tpr)
print(thresholds)

[0.         0.         0.         0.07142857 0.07142857 0.16666667
 0.16666667 0.57142857 0.61904762 1.        ]
[0.         0.01388889 0.95833333 0.95833333 0.98611111 0.98611111
 1.         1.         1.         1.        ]
[       inf 0.74945134 0.52535911 0.4598666  0.45630642 0.43730569
 0.42932512 0.32936158 0.32884527 0.28120847]


In [58]:
# AUC
AUC = auc(fpr, tpr)
AUC

0.9957010582010583

<br>

#### Q.
사이킷런 패키지 내 iris 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 AdaBoostClassifier()를 통해 다지분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 macro f1-score 사용하기)

In [59]:
# 클래스 함수 불러오기
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split

In [60]:
# iris 데이터 가져오기
from sklearn.datasets import load_iris
iris = load_iris()
data = iris.data
target = iris.target

In [61]:
# 데이터 분할하기
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2202)

In [65]:
# 모델객체 생성
ada_multi = AdaBoostClassifier(n_estimators = 100,
                            learning_rate = 0.5,
                            random_state = 2022)

In [67]:
# 모델학습
model_ada_multi = ada_multi.fit(X_train, y_train)

In [68]:
# macro f1-score
from sklearn.metrics import f1_score
y_pred = model_ada_multi.predict(X_test)

In [69]:
macro_f1 = f1_score(y_test, y_pred, average = 'macro')
macro_f1

0.899749373433584

<br><br>

#### **3-2) AdaBoostRegressor**

- 연속형 타겟변수
- ```sklearn.ensemble.AdaBoostRegressor(base_estimator = None, n_estimators = 50, learning_rate = 1.0, loss = 'linear', ...)```
   - **base_estimator**: 부스팅할 앙상블의 기본 분류기. None(default)면 DecisionTreeRegressor(max_depth = 3)
   - **n_estimators**: 트리의 수 (default = 5)
   - **learning_rate**: 부스팅 반복시 각 분류기에 적용되는 가중치 (default = 1.0)
   - **loss**: 부스팅 반복시 가중치 갱신 시 사용될 손실함수
      - 'linear' (default)
      - 'square'
      - 'exponential'

#### Q.
사이킷런 패키지 내 diabetes 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 AdaBoostRegresor()를 통해 연속형 예측 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고 평가지표는 RMSE 사용하기)

In [1]:
# 클래스 함수 불러오기
from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split

In [3]:
# diabetes 데이터 가져오기
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = diabetes.data
target = diabetes.target

In [4]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    random_state = 2205)

In [6]:
# 모델객체 생성
ada_conti = AdaBoostRegressor(n_estimators = 500,
                              learning_rate = 0.01,
                              loss = 'square',
                              random_state = 2022)

In [7]:
# 모델학습
model_ada_conti = ada_conti.fit(X_train, y_train)

In [8]:
# RMSE
from sklearn.metrics import mean_squared_error
y_pred = model_ada_conti.predict(X_test)

In [9]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
rmse

62.219496081450266

<br><br><br>
<hr>

#### **4) Gradient Boosting**

- Gradient Boosting은 사이킷런 서브패키지 ```ensemble``` 내 클래스 ```GradientBoostingClassifier()```, ```GradientBoostingRegressor()``` 등을 통해 모형객체를 생성할 수 있음
- GradientBoostingClassifier는 이진분류와 다지분류, GradientBoostingRegressor는 연속형 타겟변수에 사용됨  

#### **4-1) GradientBoostingClassifier**

- 이진분류와 다지분류
- ```sklearn.ensemble.GradientBoostingClassifier(n_estimators = 50, criterion = 'friedman_mse', max_depth = 3, min_samples_leaf = 1, max_feautres = None, learning_rate = 1.0, loss = 'log_loss',  ...)```  
   - **n_estimators**: 수행할 부스팅 단계 수. 그래디언트 부스팅은 일반적으로 숫자가 많을수록 성능이 향상됨 (default = 100)
   - **criterion**: 분할에 기준이 되는 지표
      - 'friedman_mse' (default)
      - 'squared_error'
      - 'mse'
   - **max_depth**: 트리의 최대 깊이 (default = 3)
   - **min_samples_leaf**: 리프 노드에 있어야하는 최소 샘플 수 (default = 1)
   - **max_features**: 최적의 분할을 찾을 때 고려할 컬럼 수
      - 'auto'
      - 'sqrt'
      - 'log2'
      - None (default)
   - **learning_rate**: 부스팅 반복시 각 분류기에 적용되는 가중치 (default = 1.0)
   - **loss**: 최적화할 손실함수
      - 'log_loss' (default)
      - 'deviance'
      - 'exponential'

<br>

#### Q.
사이킷런 패키지 내 breast_cancer 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 GradientBoostingClassifier()를 통해 이진분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 AUC 사용하기)

In [10]:
# 클래스 함수 가져오기
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [11]:
# breast_cancer 데이터 가져오기
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [12]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2205)

In [14]:
# 모델객체 생성
gradient_bin = GradientBoostingClassifier(n_estimators = 500,
                                          max_depth = 4,
                                          min_samples_leaf = 10,
                                          learning_rate = 0.1,
                                          random_state = 2022)

In [15]:
# 모델학습
model_gradient_bin = gradient_bin.fit(X_train, y_train)

In [16]:
# ROC
from sklearn.metrics import roc_curve, auc
y_score = model_gradient_bin.predict_proba(X_test)[:, 1]

In [17]:
fpr, tpr, thresholds = roc_curve(y_test, y_score)
print(fpr)
print(tpr)
print(thresholds)

[0.         0.         0.         0.02380952 0.02380952 0.04761905
 0.04761905 1.        ]
[0.         0.01388889 0.95833333 0.95833333 0.98611111 0.98611111
 1.         1.        ]
[           inf 1.00000000e+00 8.67972695e-01 6.29254749e-02
 2.37413316e-03 1.47966965e-03 6.74099220e-04 4.89679482e-13]


In [18]:
# AUC
AUC = auc(fpr, tpr)
AUC

0.9986772486772486

<br>

#### Q.
사이킷런 패키지 내 iris 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 GradientBoostingClassifier()를 통해 다지분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 macro f1-score 사용하기)

In [19]:
# 클래스 함수 불러오기
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split

In [20]:
# iris 데이터 가져오기
from sklearn.datasets import load_iris
iris = load_iris()
data = iris.data
target = iris.target

In [21]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2205)

In [22]:
# 모델객체 생성
gradient_multi = GradientBoostingClassifier(n_estimators = 500,
                                            max_depth = 8,
                                            min_samples_leaf = 5,
                                            learning_rate = 0.5,
                                            random_state = 2022)

In [23]:
# 모델학습
model_gradient_multi = gradient_multi.fit(X_train, y_train)

In [25]:
# macro f1-score
from sklearn.metrics import f1_score
y_pred = model_gradient_multi.predict(X_test)

In [26]:
macro_f1 = f1_score(y_test, y_pred, average = 'macro')
macro_f1

0.9665831244778613

<br><br>

#### **4-2) GradientBoostingRegressor**

- 연속형 타겟변수
- ```sklearn.ensemble.GradientBoostingRegressor(n_estimators = 100, criterion = 'friedman_mse', max_depth = 3, min_samples_leaf = 1, max_feautres = None, learning_rate = 1.0, loss = 'squared_error',  ...)```  
   - **n_estimators**: 수행할 부스팅 단계 수. 그래디언트 부스팅은 일반적으로 숫자가 많을수록 성능이 향상됨 (default = 100)
   - **criterion**: 분할에 기준이 되는 지표
      - 'friedman_mse' (default)
      - 'squared_error'
      - 'mse'
   - **max_depth**: 트리의 최대 깊이 (default = 3)
   - **min_samples_leaf**: 리프 노드에 있어야하는 최소 샘플 수 (default = 1)
   - **max_features**: 최적의 분할을 찾을 때 고려할 컬럼 수
      - 'auto'
      - 'sqrt'
      - 'log2'
      - None (default)
   - **learning_rate**: 부스팅 반복시 각 분류기에 적용되는 가중치 (default = 1.0)
   - **loss**: 최적화할 손실함수
      - 'log_loss' (default)
      - 'deviance'
      - 'exponential'

#### Q.
사이킷런 패키지 내 diabetes 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 GradientBoostingRegressor()를 통해 연속형 예측 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고 평가지표는 RMSE 사용하기)

In [27]:
# 클래스 함수 불러오기
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [28]:
# diabetes 데이터 가져오기
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = diabetes.data
target = diabetes.target

In [29]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    random_state = 2205)

In [30]:
# 모델객체 생성
gradient_conti = GradientBoostingRegressor(n_estimators = 500,
                                           max_depth = 2,
                                           min_samples_leaf = 5,
                                           learning_rate = 0.5, 
                                           random_state = 2022)

In [31]:
# 모델학습
model_gradient_conti = gradient_conti.fit(X_train, y_train)

In [32]:
# RMSE
from sklearn.metrics import mean_squared_error
y_pred = model_gradient_conti.predict(X_test)

In [33]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
rmse

75.2312380664441

<br><br><br>
<hr>

#### **5) XGBoost**

- XGBoost는 ```xgboost``` 패키지를 설치한 후 해당 사이킷런 래퍼(Wrapper) 패키지 내 클래스 ```XGBClassifier```, ```XGBRegressor``` 등을 통해 모형객체를 생성함.
- XGBClassifier는 이진분류와 다지분류, XGBRegressor는 연속형 타겟변수에 사용됨  

#### **5-1) XGBClassifier**

- 이진분류와 다지분류
- ```xgboost.XGBClassifier(max_depth = 6, n_estimators = 100, nthread = None, min_child_weight = 1, gamma = 0, objective = multi:softmax)```  
   - **max_depth**: 트리의 최대 깊이 (default = 6)
   - **n_estimators**: 수행할 부스팅 단계 수 (default = 100)
   - **n_thread**: 병렬 thread 수 (default = None)
   - **min_child_weight**: 하위 노드에 필요한 인스턴스 가중치의 최소 합계 (default = 1)
   - **gamma**: 트리의 리프노드에 추가 파티션을 만드는데 필요한 최소 손실 함수 (defualt = 0)
   - **objective**: 목푯값에 따른 학습 목적
      - 'binary:logistic'
      - 'multi:softmax'

<br>

#### Q.
사이킷런 패키지 내 breast_cancer 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 XGBClassifier()를 통해 이진분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 AUC 사용하기)

In [38]:
# 클래스, 함수 불러오기
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier

In [39]:
# breast_cancer 데이터 가져오기
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [40]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2205)

In [41]:
# 모형객체 생성
xgb_wrap_bin = XGBClassifier(max_depth = 8,
                             n_estimators = 500,
                             nthread = 5,
                             min_child_weight = 20,
                             gamma = 0.5,
                             objective = 'binary:logistic',
                             use_level_encoder = False,
                             random_state = 2022)

In [42]:
# 모델학습
model_xgb_bin = xgb_wrap_bin.fit(X_train, y_train, eval_metric = 'mlogloss')

C:\Users\kbr_u\anaconda3\Lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[02:11:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "use_level_encoder" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




- xgboost 모형은 자동으로 라벨인코딩이 수행(use_label_encoder의 dafault 값이 True)되는데 현재 데이터가 라벨인코딩이 되어있는 형태이므로 False 옵션을 추가함
- 참고) 모형객체의 메소드 ```fit()``` 내 ```eval_metric``` 인자는 학습 모형에 대한 검증시 사용되는 평가지표로 default 값은 'rmse'(연속형), 'error'(이진분류)임
- 참고) objective에 따라 사용할 수 있는 함수들이 다르고 eval_metric에 사용 가능한 함수는 ```rmse```, ```mae```, ```logloss```(negative log-likelihood), ```error```(binary classification error rate), ```merror```, ```mlogloss```(multiclass logloss), ```auc``` 등이 있음

In [43]:
# ROC
from sklearn.metrics import roc_curve, auc
y_score = model_xgb_bin.predict_proba(X_test)[:, 1]

In [45]:
fpr, tpr, thresholds = roc_curve(y_test, y_score)
print(fpr)
print(tpr)
print(thresholds)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.02380952
 0.02380952 0.02380952 0.0952381  0.11904762 0.11904762 0.14285714
 0.16666667 0.21428571 0.23809524 1.        ]
[0.         0.23611111 0.27777778 0.29166667 0.31944444 0.40277778
 0.41666667 0.47222222 0.5        0.63888889 0.68055556 0.70833333
 0.75       0.77777778 0.80555556 0.83333333 0.91666667 0.91666667
 0.94444444 0.97222222 0.97222222 0.97222222 0.98611111 1.
 1.         1.         1.         1.        ]
[       inf 0.964954   0.95908093 0.95891404 0.95740104 0.9520791
 0.9501267  0.9494602  0.94323325 0.9405234  0.92814016 0.9212042
 0.91090393 0.8974478  0.883159   0.86612844 0.5465257  0.5057164
 0.49900016 0.4262316  0.3945671  0.3077966  0.2066235  0.19412422
 0.17169775 0.14115652 0.10407268 0.07343688]


In [46]:
# AUC
AUC = auc(fpr, tpr)
AUC

0.9952050264550264

<br>

#### Q.
사이킷런 패키지 내 iris 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 XGBClassifier()를 통해 다지분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 macro f1-score 사용하기)

In [47]:
# 클래스 함수 불러오기
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split

In [48]:
# iris 데이터 가져오기
from sklearn.datasets import load_iris
iris = load_iris()
data = iris.data
target = iris.target

In [49]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target,
                                                    test_size = 0.2,
                                                    random_state = 2205,
                                                    stratify = target)

In [50]:
# 모델객체 생성
xgb_multi = XGBClassifier(max_depth = 8,
                          n_estimators = 500,
                          nthread = 5,
                          min_child_weight = 10,
                          gamma = 0.5,
                          objective = 'multi:softmax',
                          use_label_encoder = False,
                          random_state = 2022)

In [51]:
# 모델학습
model_xgb_multi = xgb_multi.fit(X_train, y_train, eval_metric = 'mlogloss')

In [57]:
# macro f1-score
from sklearn.metrics import f1_score
y_pred = model_xgb_multi.predict(X_test)

In [58]:
macro_f1 = f1_score(y_test, y_pred, average = 'macro')
macro_f1

0.9665831244778613

<br><br>
      
#### **5-2) XGBRegressor**

- 연속형 타겟변수
- ```xgboost.XGBRegressor(max_depth = 6, n_estimators = 100, nthread = None, min_child_weight = 1, gamma = 0, objective = reg:squarederror)```  
   - **max_depth**: 트리의 최대 깊이 (default = 6)
   - **n_estimators**: 수행할 부스팅 단계 수 (default = 100)
   - **n_thread**: 병렬 thread 수 (default = None)
   - **min_child_weight**: 하위 노드에 필요한 인스턴스 가중치의 최소 합계 (default = 1)
   - **gamma**: 트리의 리프노드에 추가 파티션을 만드는데 필요한 최소 손실 함수 (defualt = 0)
   - **objective**: 목푯값에 따른 학습 목적. 회귀는 'reg:squarederror'

#### Q.
사이킷런 패키지 내 diabetes 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 XGBRegressor()를 통해 연속형 예측 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고 평가지표는 RMSE 사용하기)

In [59]:
# 클래스, 함수 불러오기
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [60]:
# diabetes 데이터 가져오기
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = diabetes.data
target = diabetes.target

In [63]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data, 
                                                    target, 
                                                    test_size = 0.2,
                                                    random_state = 2205)

In [64]:
# 모델객체 생성
xgb_conti = XGBRegressor(max_depth = 8,
                         n_estimators = 500,
                         n_thread = 5,
                         min_child_weight = 10,
                         gamma = 0.5,
                         objective = 'reg:squarederror',
                         use_label_encoder = False,
                         random_state = 2022)

In [65]:
# 모델학습
model_xgb_conti = xgb_conti.fit(X_train, y_train, eval_metric = 'rmse')

[02:57:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "n_thread" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




In [66]:
# RMSE
from sklearn.metrics import mean_squared_error
y_score = model_xgb_conti.predict(X_test)

In [67]:
rmse = mean_squared_error(y_test, y_score, squared = False)
rmse

67.9309022195871

<br><br><br>
<hr>

#### **6) LightGBM**

- LightGBM은 ```lightgbm``` 패키지를 설치한 후 해당 사이킷런 래퍼(Wrapper) 패키지 내 클래스 ```LGBMClassifier```, ```LGBMRegressor``` 등을 통해 모형객체를 생성함.
- LGBMClassifier는 이진분류와 다지분류, LGBMRegressor는 연속형에 사용됨  

#### **6-1) LGBMClassifier**

- 이진분류와 다지분류
- ```lightgbm.LGBMClassifier(boosting_type = 'gbdt', max_depth = -1, n_estimators = 100, n_jobs = None, learning_rate = 0.1, objective = None, ...)```
   - **boosting_type**: 실행하고자 하는 알고리즘 타입
      - 'gbdt': Traditional Gradient Boosting Decision Tree (default)
      - 'rf': Random Forest
   - **max_depth**: 트리의 최대 깊이, <=0은 제한을 두지 않는 것을 의미 (default = -1)
   - **n_estimators**: 수행할 부스팅 단계 수 (default = 100)
   - **n_jobs**: 병렬 thread 수 (default = None)
   - **learning_rate**: 부스팅 반복시 각 분류기에 적용되는 가중치 (default = 0.1)
   - **objective**: 목푯값에 따른 학습 목적
      - 'binary'
      - 'multiclass'

<br>

#### Q.
사이킷런 패키지 내 breast_cancer 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 LGBMClassifier()를 통해 이진분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 AUC 사용하기)

In [69]:
!pip install lightgbm==3.3.2

  Obtaining dependency information for lightgbm==3.3.2 from https://files.pythonhosted.org/packages/ba/24/2f83a1008c8add8cd9da03163f911be6b555eb2b9166b5ab74e1ad63ff40/lightgbm-3.3.2-py3-none-win_amd64.whl.metadata
   ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
   ---------------------------------------  1.0/1.0 MB 32.4 MB/s eta 0:00:01
   ---------------------------------------- 1.0/1.0 MB 12.9 MB/s eta 0:00:00


In [72]:
# 클래스 함수 불러오기
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split

In [73]:
# breast_cancer 데이터 가져오기
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [74]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2205)

In [75]:
# 모델객체 생성
lgbm_bin = LGBMClassifier(max_depth = 8,
                          n_estimators = 500,
                          n_jobs = 30,
                          min_child_weight = 10,
                          learning_rate = 0.2,
                          objective = 'binary',
                          random_state = 2022)

In [76]:
# 모델학습
model_lgbm_bin = lgbm_bin.fit(X_train, y_train)

In [77]:
# ROC
from sklearn.metrics import roc_curve, auc
y_pred = model_lgbm_bin.predict_proba(X_test)[:, 1]

In [78]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
print(fpr)
print(tpr)
print(thresholds)

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.02380952
 0.02380952 0.23809524 0.28571429 0.80952381 0.92857143 1.        ]
[0.         0.01388889 0.125      0.15277778 0.31944444 0.34722222
 0.43055556 0.45833333 0.73611111 0.76388889 0.98611111 0.98611111
 1.         1.         1.         1.         1.         1.        ]
[       inf 0.99542067 0.99426807 0.99420371 0.99198285 0.99137414
 0.99065517 0.99063428 0.95378065 0.95187075 0.48683936 0.33081646
 0.32117036 0.03376842 0.02998259 0.01030292 0.0073062  0.00560571]


In [79]:
# AUC
AUC = auc(fpr, tpr)
AUC

0.9996693121693121

<br>

#### Q.
사이킷런 패키지 내 iris 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 LGBMClassifier()를 통해 다지분류 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고, target의 비율을 반영하고 평가지표는 macro f1-score 사용하기)

In [80]:
# 클래스 함수 불러오기
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier 

In [81]:
# iris 데이터 가져오기
from sklearn.datasets import load_iris
iris = load_iris()
data = iris.data
target = iris.target

In [82]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    stratify = target,
                                                    random_state = 2205)

In [83]:
# 모형객체 생성
lgbm_multi = LGBMClassifier(max_depth = 8,
                            n_estimators = 500,
                            n_jobs = 5,
                            min_child_weight = 10,
                            learning_rate = 0.5,
                            objective = 'multiclass',
                            random_state = 2022)

In [84]:
# 모델학습
model_lgbm_multi = lgbm_multi.fit(X_train, y_train)

In [85]:
# macro f1-score
from sklearn.metrics import f1_score
y_score = model_lgbm_multi.predict(X_test)

In [86]:
macro_f1 = f1_score(y_test, y_score, average = 'macro')
macro_f1

0.9665831244778613

<br><br>
      
#### **5-2) XGBRegressor**

- 연속형 타겟변수
- ```lightgbm.LGBMRegressor(boosting_type = 'gbdt', max_depth = -1, n_estimators = 100, n_jobs = None, learning_rate = 0.1, objective = None, ...)```
   - **boosting_type**: 실행하고자 하는 알고리즘 타입
      - 'gbdt': Traditional Gradient Boosting Decision Tree (default)
      - 'rf': Random Forest
   - **max_depth**: 트리의 최대 깊이, <=0은 제한을 두지 않는 것을 의미 (default = -1)
   - **n_estimators**: 수행할 부스팅 단계 수 (default = 100)
   - **n_jobs**: 병렬 thread 수 (default = None)
   - **learning_rate**: 부스팅 반복시 각 분류기에 적용되는 가중치 (default = 0.1)
   - **objective**: 목푯값에 따른 학습 목적
      - 회귀는 'regression'

#### Q.
사이킷런 패키지 내 diabetes 데이터를 호출한 후 학습 데이터와 평가 데이터로 분할하고 클래스 LGBMRegresor()를 통해 연속형 예측 모형객체를 생성하고 학습한 후 평가 데이터로 목표값을 예측하고 성능을 측정하는 코드 작성하기 (단, 학습과 평가 데이터의 비율은 8:2로 하고 평가지표는 RMSE 사용하기)

In [87]:
# 클래스 함수 불러오기
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split

In [89]:
# diabetes 데이터 가져오기
from sklearn.datasets import load_diabetes
diabetes = load_diabetes()
data = diabetes.data
target = diabetes.target

In [90]:
# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data,
                                                    target,
                                                    test_size = 0.2,
                                                    random_state = 2205)

In [91]:
# 모델객체 생성
lgbm_conti = LGBMRegressor(max_depth = 8,
                           n_estimators = 500,
                           n_jobs = 5,
                           min_child_weight = 10,
                           learning_rate = 0.5,
                           objective = 'regression',
                           random_state = 2022)

In [92]:
# 모델학습
model_lgbm_conti = lgbm_conti.fit(X_train, y_train)

In [93]:
# RMSE
from sklearn.metrics import mean_squared_error
y_pred = model_lgbm_conti.predict(X_test)

In [94]:
rmse = mean_squared_error(y_test, y_pred, squared = False)
rmse

63.62631474228169